# Section 3.3: Automate Test Generation with Prompt Templates

| **Aspect** | **Details** |
|-------------|-------------|
| **Goal** | Build a template that transforms vague requirements into comprehensive test specifications |
| **Time** | ~40 minutes |
| **Prerequisites** | Sections 3.1-3.2 complete, `setup_utils.py` loaded |
| **What You'll Build** | Requirements analyzer with ambiguity detection and edge case generation |
| **Next Steps** | Activity 3.3 → **Core module complete!** (Optional: Section 3.4 Advanced) |

---

You built a code review template in Section 3.2. Now tackle the **testing phase**: create a prompt that detects ambiguities, generates edge cases, and separates unit from integration tests—turning "build a discount system" into actionable test plans.

## 🔧 Quick Setup Check

Since you completed Section 1, setup is already done! We just need to import it.


In [ ]:
# Quick setup check - imports setup_utils
try:
    import importlib
    import setup_utils
    importlib.reload(setup_utils)
    from setup_utils import *
    print(f"✅ Setup loaded! Using {PROVIDER.upper()} with {get_default_model()}")
    print("🚀 Ready to build test generation templates!")
except ImportError:
    print("❌ Setup not found!")
    print("💡 Please run 3.1-setup-and-introduction.ipynb first to set up your environment.")


## 🧪 Test Generation Automation Template

### Building a Comprehensive Test Generation Prompt with a Multi-Tactic Stack

<div style="background:#fef3c7; border-left:4px solid #f59e0b; padding:16px; border-radius:6px; margin:20px 0; color:#000000;">
<strong style="color:#92400e;">🎯 What You'll Build in This Section</strong><br><br>

You'll create a <strong>production-ready test generation prompt template</strong> that analyzes vague requirements, surfaces coverage gaps, and produces reusable specs across unit and integration tests.

<strong>Time Required:</strong> ~35 minutes (learning + examples + activity)
</div>

Test generation makes the model juggle requirements, existing coverage, and missing scenarios at the same time. We'll reuse Module 2 tactics so the flow moves from context → analysis → gap filling without losing track of dependencies.

#### 🎯 The Problem We're Solving

Manual test planning faces three critical challenges:

1. **📋 Incomplete Coverage**
   - Easy to miss edge cases and error paths
   - Boundary conditions often overlooked (0%, 100%, empty inputs)
   - Security and performance scenarios fall through the cracks
   - **Impact:** Bugs slip through to production and erode trust

2. **⏰ Time Pressure**
   - Testing gets squeezed at the end of sprints
   - QA teams struggle to keep up with feature velocity
   - Documentation for test planning is often rushed or skipped
   - **Impact:** Technical debt builds up inside the test suite

3. **🎲 Missed Ambiguities**
   - Unclear requirements don't get questioned until implementation
   - Assumptions creep in without validation
   - Integration points and dependencies surface late
   - **Impact:** Rework, missed deadlines, and scope creep


#### 🧩 Pattern Overview: AWS-Inspired Flow

The **[AWS `generate-tests` command pattern](https://github.com/aws-samples/anthropic-on-aws/blob/main/advanced-claude-code-patterns/commands/generate-tests.md)** distills test planning into four focused moves. Keeping each move small reduces cognitive load while preserving coverage rigor:

1. **Command Summary** – clarify the mission, the success signals, and who consumes the plan.
2. **System Inputs** – bundle the minimal context (domain, requirements, existing tests) so the model scans once.
3. **Reasoning Checklist** – guide how the model should think before it writes any specs.
4. **Output Contract** – lock the deliverables into a predictable structure for downstream tooling.

<div style="background:#dcfce7; border-left:4px solid #22c55e; padding:16px; border-radius:6px; margin:20px 0; color:#000000;">
<strong style="color:#166534;">AWS-Inspired Callouts</strong><br>
<ul style="margin:12px 0 0 18px;">
  <li>Keep the command summary to three bullets so the north star stays visible.</li>
  <li>Group all inputs together to avoid jumping between sections.</li>
  <li>Use a checklist—short, ordered steps—to enforce deliberate reasoning.</li>
  <li>Let the output contract mirror the sections your QA automation expects.</li>
</ul>
</div>



### 🗂️ Command Template Walkthrough

Follow the four sections in order. Swap in your project variables and keep each bullet list tight so learners (and the model) stay oriented.

**Where the tactics show up in the template:**

| Template Block | What It Does | Tactic Used |
| --- | --- | --- |
| **`<command_summary>`** | Frames intent, consumers, and success signals | Role prompting & guardrails |
| **`<system_inputs>`** with **`<ProjectOverview>`**, **`<FunctionalRequirements>`**, **`<ExistingTests>`** | Bundles domain, requirements, and existing coverage for one-pass reading | Structured inputs |
| **`<reasoning_checklist>`** | Forces deliberate analysis before drafting specs | Task decomposition + chain-of-thought |
| **`<output_contract>`** | Locks deliverables into an automation-friendly format | Structured output |

Use these four tags to tune the prompt for any feature: update `<command_summary>` with your mission and success signals, fill `<system_inputs>` with the latest project context, sharpen `<reasoning_checklist>` questions to match risk areas, and rework `<output_contract>` sections so the output lands exactly where your QA tooling expects.

```xml
<!-- Section 1: Command Summary -->
<command_summary>
Command: Generate a coverage-focused test plan before sprint planning.
Primary Objective: Expose untested scenarios so the QA team can prioritise automation.
Success Signals:
- Each critical flow has at least one unit or integration test candidate.
- Ambiguities and blockers are captured for follow-up.
- Output structure stays automation-ready for QA tooling.
</command_summary>

<!-- Section 2: System Inputs -->
<system_inputs>
<ProjectOverview>
{project_context}
</ProjectOverview>
<FunctionalRequirements>
{functional_requirements}
</FunctionalRequirements>
<ExistingTests>
{existing_tests}
</ExistingTests>
</system_inputs>

<!-- Section 3: Reasoning Checklist -->
<reasoning_checklist>
1. Summarise the product slice in two bullets to anchor context.
2. Compare requirements against existing tests and note risk themes.
3. Log ambiguities or missing business rules that block automation.
4. Expand uncovered scenarios into test specifications (unit or integration) with setup, steps, and expected results before writing the final output.
</reasoning_checklist>

<!-- Section 4: Output Contract -->
<output_contract>
## Summary
- Product goal
- High-risk areas

## Ambiguities & Follow-ups
- [Question]
- [Question]

## Coverage Map
| Theme | Risk Level | Missing Scenario |
| --- | --- | --- |

## Unit Tests
### Test: [Name]
**Goal:** [Purpose]
**Setup:** [Data, mocks]
**Steps:**
1. ...
**Expected:** ...

## Integration Tests
### Test: [Name]
**Goal:** [Purpose]
**Setup:** [Services, data]
**Steps:**
1. ...
**Expected:** ...

## Test Data & Tooling
- Fixtures, environments, monitoring hooks required.
</output_contract>
```

This command template keeps the analysis, gap spotting, and deliverables in one tight flow. Learners can adapt it by swapping out the variables while the structure stays stable.








### 💻 Working Example: Payment Service Test Generation

Now let's watch the command-style template drive a full test planning session for a payment processing service.

**What to look for:**
- Each section is labelled (`<!-- Section 1: ... -->`) so you can map it back to the walkthrough table.
- The reasoning checklist forces the model to analyse coverage gaps before it drafts specs.
- Unit and integration tests remain separated, and infrastructure needs stay visible at the end.

Run the cell below to see the prompt and response rendered together.



In [ ]:
# Example: Test Case Generation for Payment Service

functional_requirements = """
Payment Processing Requirements:
1. Process credit card payments with validation
2. Handle multiple currencies (USD, EUR, GBP)
3. Apply discounts and calculate tax
4. Generate transaction receipts
5. Handle payment failures and retries (max 3 attempts)
6. Send confirmation emails on success
7. Log all transactions for audit compliance
8. Support payment refunds within 30 days
"""

existing_tests = """
Current Test Suite (payment_service_test.py):
- test_process_valid_payment() - Happy path for USD payments
- test_invalid_card_number() - Validates card number format
- test_calculate_tax() - Tax calculation for US region only
"""

project_context = """
Domain: FinTech payments platform
Project: Payment Processing Service
Primary test framework: pytest
Tech stack: Python, FastAPI, PostgreSQL
"""

command_prompt = f"""
<!-- Section 1: Command Summary -->
<command_summary>
Command: Generate a coverage-focused test plan for the payment processing service.
Primary Objective: Expose untested scenarios before sprint planning.
Success Signals:
- Every critical flow has at least one unit or integration test candidate.
- Ambiguities and open questions are captured for follow-up.
- Output structure stays automation-ready for QA tooling.
</command_summary>

<!-- Section 2: System Inputs -->
<system_inputs>
<ProjectOverview>
{project_context}
</ProjectOverview>
<FunctionalRequirements>
{functional_requirements}
</FunctionalRequirements>
<ExistingTests>
{existing_tests}
</ExistingTests>
</system_inputs>

<!-- Section 3: Reasoning Checklist -->
<reasoning_checklist>
1. Summarise the product slice in two bullets to anchor context.
2. Compare requirements against existing tests and note risk themes.
3. Log ambiguities or missing business rules that block automation.
4. Expand uncovered scenarios into test specifications (unit or integration) with setup, steps, and expected results before writing the final output.
</reasoning_checklist>

<!-- Section 4: Output Contract -->
<output_contract>
## Summary
- Product goal
- High-risk areas

## Ambiguities & Follow-ups
- [Question]
- [Question]

## Coverage Map
| Theme | Risk Level | Missing Scenario |
| --- | --- | --- |

## Unit Tests
### Test: [Name]
**Goal:** [Purpose]
**Setup:** [Data, mocks]
**Steps:**
1. ...
**Expected:** ...

## Integration Tests
### Test: [Name]
**Goal:** [Purpose]
**Setup:** [Services, data]
**Steps:**
1. ...
**Expected:** ...

## Test Data & Tooling
- Fixtures, environments, monitoring hooks required.
</output_contract>
"""

test_messages = [
    {
        "role": "system",
        "content": "You follow structured QA templates and produce detailed, automation-ready test plans."
    },
    {
        "role": "user",
        "content": command_prompt
    }
]

print("📋 PROMPT PREVIEW")
print("=" * 70)
print(command_prompt)
print("=" * 70)
print("🧪 TEST GENERATION IN PROGRESS...")
print("=" * 70)
test_result = get_chat_completion(test_messages, temperature=0.0)
print(test_result)
print("=" * 70)



## 🏋️ Activity 3.3: Build Your Test Generation Template

<div style="background:#dcfce7; border-left:4px solid #22c55e; padding:16px; border-radius:6px; margin:20px 0; color:#065f46;">
<strong style="color:#166534;">🎯 Your Task:</strong> Create a test generation template for a shopping cart discount system with intentionally vague requirements.<br><br>
<strong>Time:</strong> 30-40 minutes
</div>

### 📝 How to Complete This Activity

1. Open **[`activities/activity-3.3-test-generation.md`](./activities/activity-3.3-test-generation.md)**
2. Complete the template between the `<!-- TEMPLATE START -->` and `<!-- TEMPLATE END -->` markers
3. Return here and run the test cell below
4. Compare with **[solution](./solutions/activity-3.3-test-generation-solution.md)** when done

> **💡 The Challenge:** The requirements are intentionally vague! Your template should identify ambiguities, generate edge cases, and produce comprehensive test specifications.

<div style="margin-top:16px; color:#991b1b; padding:12px; background:#fee2e2; border-radius:6px; border-left:4px solid #ef4444;">
<style>
code {
  font-family: Consolas,"courier new";
  color:rgb(238, 13, 13);
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 110%;
}
</style>
<strong>⚠️ COMPLETE THE ACTIVITY FIRST:</strong><br>
Before running the cells below, you must:
<ol style="margin: 8px 0 0 0;">
<li>Open <code>activities/activity-3.3-test-generation.md</code></li>
<li>Replace all <code>TODO</code> comments in the template (between <code>&lt;!-- TEMPLATE START --&gt;</code> and <code>&lt;!-- TEMPLATE END --&gt;</code>)</li>
<li>Save the file</li>
<li>Return here to test your template</li>
</ol>
</div>

In [ ]:
# Test your Activity 3.3 template

# Shopping Cart Discount System requirements (intentionally vague!)
discount_requirements = """
Feature: Shopping Cart Discount System

Requirements:
1. Users can apply discount codes at checkout
2. Discount types: percentage (10%, 25%, etc.) or fixed amount ($5, $20, etc.)
3. Each discount code has an expiration date
4. Usage limits: one-time use OR unlimited
5. Business rule: Discounts cannot be combined (one per order)
6. Cart total must be > 0 after discount applied
7. Fixed discounts cannot exceed cart total
"""

existing_tests = """
Current test suite (minimal coverage):
- test_apply_percentage_discount() - 10% off $100 cart
- test_fixed_amount_discount() - $5 off $50 cart
"""

# Run this to test your template from the activity file
test_activity_3_3(
    test_code=discount_requirements,
    variables={
        'domain': 'e-commerce',
        'project_name': 'Shopping Cart',
        'tech_stack': 'Python/Flask',
        'test_framework': 'pytest',
        'functional_requirements': discount_requirements,
        'test_suite_overview': existing_tests
    }
)

# The function will:
# 1. Read your template from activities/activity-3.3-test-generation.md
# 2. Substitute the variables
# 3. Send to the AI model
# 4. Display the results
# 5. Asks if you want to save results back to the activity file


### 📚 Learn More: Advanced Test Generation Patterns

Want to dive deeper into automated test generation?

**📖 AWS Anthropic Advanced Patterns:**
- [Test Generation Command Pattern](https://github.com/aws-samples/anthropic-on-aws/blob/main/advanced-claude-code-patterns/commands/generate-tests.md) - Production-ready patterns

**🔗 Related Best Practices:**
- [Claude 4 Prompt Engineering](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/claude-4-best-practices)
- [Prompt Templates and Variables](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/prompt-templates-and-variables)

## ✅ Section 3 Complete!

<div style="margin-top:16px; padding:14px; background:#dcfce7; border-left:4px solid #22c55e; border-radius:6px; color:#065f46;">
<strong>🎉 Outstanding work!</strong> You just wrapped up the Test Generation Automation section and completed the core material for Module 3!
</div>

**Key takeaways**
- Crafted a reusable template that analyzes requirements, finds coverage gaps, and plans infrastructure
- Practiced separating unit and integration specs while documenting assumptions
- Reinforced the Module 2 tactic stack inside a real SDLC workflow

**Where to go next**
1. Complete the "Track Your Progress" section below to assess your skills
2. Optional: Continue to **Section 3.4: LLM-as-Judge** for advanced evaluation techniques
3. Revisit your activity files to iterate on the templates with real project requirements
4. Share what you built pair with teammates or incorporate into CI to keep momentum going

<div style="margin-top:12px; padding:24px; background:linear-gradient(120deg,#1d4ed8 0%,#6366f1 50%,#22d3ee 100%); border-radius:14px; color:#f8fafc; text-align:center; box-shadow:0 10px 24px rgba(79,70,229,0.25);">
  <strong style="display:block; font-size:1.15em; margin-bottom:6px;">☕ Time for a quick check-in?</strong>
  <span style="font-size:0.98em; line-height:1.6;">Scroll down to track your progress before moving to advanced material.</span>
</div>

## 📈 Track Your Progress

> **💡 New to Skills Checklists?** These checkboxes help you track mastery of specific, actionable skills. Check them off when you can confidently apply each skill to your own projects.

### Self-Assessment Questions

After completing Module 3 core sections (3.1-3.3), ask yourself:

1. Can I design a multi-dimensional code review template that catches security, performance, and quality issues?
2. Can I create parameterized prompt templates that work across different services and tech stacks?
3. Can I identify ambiguities in vague requirements and expand them into test specifications?
4. Can I structure prompts to separate unit tests from integration tests?
5. Can I validate and iterate on prompt templates using test scenarios?
6. Can I adapt AWS patterns to my team's specific SDLC workflows?

### Progress Overview

<div style="margin-bottom:12px; padding:10px; background:#fef3c7; border-left:4px solid #f59e0b; border-radius:4px; color:#78350f;">
<strong>💡 Note:</strong> The status indicators below (✅/⬜) are <strong>visual guides only</strong> and cannot be clicked. Scroll down to "<strong>Check Off Your Skills</strong>" for the interactive checkboxes where you'll track your actual progress!
</div>

<div style="background:#f8f9fa; border-radius:8px; padding:18px 22px; margin-bottom:18px; border:1px solid #e0e0e0; box-shadow:0 1px 4px #0001; color:#000000">

<span style="color:#000000; font-weight:600">**Module 3 Core Skills Checklist:**</span>
<div style="margin:10px 0; font-size:0.9em; color:#666">Track your progress by checking off skills below. Master all 6 core skills for 100% completion!</div>

**Core Path Status:**
- <span style="color:#059669">✅ Section 3.1 Complete (Setup & Introduction)</span>
- <span style="color:#059669">✅ Section 3.2 Complete (Code Review Automation)</span>
- <span style="color:#059669">✅ Section 3.3 Complete (Test Generation Automation)</span>
- <span style="color:#e97316">⬜ Core Skills Mastery (Use Checklist Below)</span>

**Progress Guide:**
- <span style="color:#666; font-size:0.9em">0-2 skills checked: Beginner (17-42%)</span>
- <span style="color:#666; font-size:0.9em">3-4 skills checked: Intermediate (58-75%)</span>
- <span style="color:#666; font-size:0.9em">5-6 skills checked: Advanced (92-100%) 🎉</span>

</div>

### Check Off Your Core Skills

<div style="background:#f8f9fa; border-radius:8px; padding:18px 22px; margin-bottom:18px; border:1px solid #e0e0e0; box-shadow:0 1px 4px #0001; color:#000000">

<span style="color:#000000">Mark each skill as you master it:</span>

<span style="color:#000000; font-weight:600">**Section 3.2: Code Review Automation**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill1" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can design multi-dimensional code review templates (security, performance, quality)</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill2" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can use parameterized templates with {{variables}} for reusable workflows</span>
</div>

<span style="color:#000000; font-weight:600">**Section 3.3: Test Generation Automation**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill3" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can identify ambiguities in requirements and generate clarifying questions</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill4" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can expand requirements into comprehensive test specifications with edge cases</span>
</div>

<span style="color:#000000; font-weight:600">**General Application Skills**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill5" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can validate and iterate on prompt templates using test scenarios</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill6" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can adapt industry patterns (AWS, Anthropic) to my team's SDLC workflows</span>
</div>

</div>

<div style="margin-top:16px; padding:12px; background:#dbeafe; border-radius:6px; border-left:4px solid #3b82f6; color:#000000">
<strong>💡 Ready for Advanced Material?</strong> If you've checked off all 6 core skills and want to deepen your expertise, continue to <a href="./3.4-llm-as-judge-evaluation.ipynb">Section 3.4: LLM-as-Judge Evaluation</a> to learn rubric-based evaluation and systematic testing.
</div>

🎉 **Module 3 Core Complete!** You've learned to automate code reviews and test generation with reusable prompt templates.

## 🎊 What You've Accomplished

- ✅ **Built parameterized code review templates** that catch security, performance, and quality issues
- ✅ **Created test generation workflows** that identify ambiguities and expand edge cases
- ✅ **Applied Module 2 tactics** to real SDLC scenarios (role prompting, structured inputs, task decomposition)
- ✅ **Tested and refined templates** using automated helper functions
- ✅ **Adapted industry patterns** from AWS and Anthropic for your workflows

### Next Steps

**Option 1: Apply What You've Learned (Recommended)**
- Use your templates on real project code
- Integrate into CI/CD pipelines
- Share with your team and gather feedback

**Option 2: Master Advanced Evaluation (Optional)**
Continue to **[Section 3.4: LLM-as-Judge Evaluation](./3.4-llm-as-judge-evaluation.ipynb)** to learn:
- Weighted rubric design for automated quality gates
- Systematic evaluation with production platforms
- How to scale from manual judging to automated decisions

**Option 3: Integrate with AI Code Assistants (Module 4)**
Continue to **Module 4: Integration** to learn how to:
- Integrate your prompt templates into GitHub Copilot, OpenAI Codex, and Claude Code
- Build custom commands and workflows for your AI coding assistant
- Operationalize prompt engineering across your development team